In [1]:
import numpy as np

# Anisotropic scattering under stationary conditions
General solution
\begin{align}
\textbf{E}&=\sum_{i=1}^2c_i\textrm{e}^{\lambda_i (x-x_i)}\textbf{v}_i
\end{align}
Boundary conditions:
\begin{align}
1&=\sum_{i=0}^1c_i\textrm{e}^{-\lambda_i x_i}\textbf{v}_{i0}\\
0&=\sum_{i=0}^1c_i\textrm{e}^{\lambda_i (w-x_i)}\textbf{v}_{i1}
\end{align}

In [2]:
class EigenSystem:
    def __init__(self, eps=0., beta=0, w=.1):
        self.eps = eps
        self.beta = beta
        self.w = w

    @property
    def matrix(self):
        a = -self.eps - .5*(1 + self.beta)
        b = - .5*(1 - self.beta)
        c = - .5*(1 + self.beta)
        d = self.eps - .5*(1 - self.beta)
        return np.array([[a,b], [c,d]])

    @property
    def eig_tuples(self):
        tups = []
        for i in range(2):
            lam = self.evals[i]
            vec = self.evecs[:,i]
            xe = 0. if lam <= 0. else self.w
            tups += [(lam, vec, xe)]
        return tups
        
    def solve(self):
        self.evals, self.evecs = np.linalg.eig(self.matrix)
        b = np.array([1.,0.])
        a = np.zeros((2,2))
        x = np.array([0., self.w])
        for i, (lam, vec, xe) in enumerate(self.eig_tuples):
            a[:,i] = vec * np.exp(lam * (x - xe))
        self.coeffs = np.linalg.solve(a, b)

    def calc_expansion(self, x):
        u = 0.
        for (lam, vec, xe), coeff in zip(self.eig_tuples, self.coeffs):
            u += coeff * np.outer(vec, np.exp(lam * (x - xe)))
        return u

## Solve and test the boundary conditions

In [3]:
es = EigenSystem()
es.solve()
x = np.array([0., es.w])
es.calc_expansion(x)

array([[ 1.00000000e+00,  9.50041625e-01],
       [ 4.99583750e-02, -5.55111512e-17]])